In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division

import os
import collections
import csv
import pickle

import numpy as np
import scipy.special
import habitat
import sklearn.neighbors

from sensei import utils

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
env_id = 'uNb9QFRL6hY'
dataset = 'mp3d'

habitat_dir = os.path.join('/Users', 'reddy', 'habitat-api')
scenes_dir = os.path.join(habitat_dir, 'data', 'scene_datasets')
path_to_cfg = os.path.join(habitat_dir, 'configs', 'datasets', 'pointnav', '%s.yaml' % dataset)
data_path = os.path.join(habitat_dir, 'data', 'datasets', 'pointnav', dataset, 'v1', '{split}', '{split}.json.gz')
scene_path = os.path.join(scenes_dir, dataset, env_id, '%s.glb' % env_id)

In [ ]:
config = habitat.get_config(path_to_cfg)

In [ ]:
config.defrost()
config.DATASET.SPLIT = 'val_mini'
config.DATASET.SCENES_DIR = scenes_dir
config.DATASET.DATA_PATH = data_path
config.SIMULATOR.SCENE = scene_path
config.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'SEMANTIC_SENSOR']
config.freeze()

In [ ]:
env = habitat.Env(config=config)

look around

In [ ]:
agent_state = env._sim.get_agent_state()

In [ ]:
sensor = env._sim.sensor_suite.sensors['semantic']

In [ ]:
rgb_obs = env._sim.get_observations_at(agent_state.position, agent_state.rotation)['rgb']

In [ ]:
plt.imshow(rgb_obs)
plt.show()

In [ ]:
scene = env.sim.semantic_annotations()

In [ ]:
sem_obs = env._sim.get_observations_at(agent_state.position, agent_state.rotation)['semantic']

In [ ]:
cat_of_mask = {idx: scene.objects[idx].category.name('') for idx in np.unique(sem_obs) if scene.objects[idx] is not None}
mask_of_cat = collections.defaultdict(list)
for k, v in cat_of_mask.items():
  if v != '':
    mask_of_cat[v].append(k)

In [ ]:
n = 50
for k, v in mask_of_cat.items():
  cat_img = np.isin(sem_obs, list(v))
  if cat_img.any():
    plt.title(k)
    cat_img = np.concatenate((rgb_obs, (n + cat_img[:, :, np.newaxis] * (255-n))), axis=2)
    plt.axis('off')
    plt.imshow(cat_img)
    plt.show()

bird's eye

In [ ]:
n_samples = 100000
samples = np.array([env._sim.sample_navigable_point() for _ in range(n_samples)])
samples = samples[:, [0, 2, 1]]

In [ ]:
plt.hist(samples[:, 2], bins=20)
plt.show()

In [ ]:
collections.Counter(samples[:, 2])

In [ ]:
min_z = -2
max_z = 1.5
mins = np.array([-100, -100])
maxs = np.array([100, 100])

In [ ]:
navigable_mask = np.ones(samples.shape[0]).astype(bool)
navigable_mask &= np.less(samples[:, 2], max_z) & np.greater(samples[:, 2], min_z)
navigable_mask &= np.less(samples[:, :2], maxs).all(axis=1) & np.greater(samples[:, :2], mins).all(axis=1)
samples = samples[navigable_mask, :]

In [ ]:
plt.scatter(samples[:, 0], samples[:, 1], c=samples[:, 2])
plt.show()

In [ ]:
cats = {obj.category.name() for obj in scene.objects if obj is not None and obj.category.name() != ''}
idx_of_cat = {x: i for i, x in enumerate(cats)}

In [ ]:
n_objects = len(idx_of_cat)
n_objects

In [ ]:
poses_of_cat = collections.defaultdict(list)
for i, obj in enumerate(scene.objects):
  if obj is None:
    continue
  cat = obj.category.name()
  if cat not in idx_of_cat:
    continue
  obj_pos = obj.aabb.center[[0, 2]]
  poses_of_cat[cat].append(obj_pos)
  sizes = obj.aabb.sizes[[0, 2]]
  for x in np.arange(obj_pos[0]-sizes[0]/2, obj_pos[0]+sizes[0]/2, 0.1):
    for y in np.arange(obj_pos[1]-sizes[1]/2, obj_pos[1]+sizes[1]/2, 0.1):
      poses_of_cat[cat].append(np.array([x, y]))
  poses_of_cat[cat].append(obj_pos)
poses_of_cat = {k: np.array(v) for k, v in poses_of_cat.items()}

In [ ]:
for cat, poses in poses_of_cat.items():
  plt.title(cat)
  plt.scatter(samples[:, 0], samples[:, 1], color='gray', alpha=0.5)
  plt.scatter(poses[:, 0], poses[:, 1], color='red', s=1)
  plt.xlim([np.min(samples[:, 0]), np.max(samples[:, 0])])
  plt.ylim([np.min(samples[:, 1]), np.max(samples[:, 1])])
  plt.show()

In [ ]:
def viz_cat_locs(cats, cat_colors):
  plt.axis('off')
  plt.scatter(samples[:, 0], samples[:, 1], color='lightgray', alpha=0.25)
  for cat, cat_color in zip(cats, cat_colors):
    poses = poses_of_cat[cat]
    plt.scatter(poses[:, 0], poses[:, 1], color=cat_color, s=1)
  plt.xlim([np.min(samples[:, 0]), np.max(samples[:, 0])])
  plt.ylim([np.min(samples[:, 1]), np.max(samples[:, 1])])
  plt.show()

In [ ]:
viz_cat_locs(['floor', 'bed'], ['gray', 'orange'])

In [ ]:
layout_path = os.path.join(utils.hab_data_dir, 'layout.pkl')

In [ ]:
with open(layout_path, 'wb') as f:
  pickle.dump((samples, poses_of_cat), f, pickle.HIGHEST_PROTOCOL)